In [25]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [2]:
infile = pd.read_csv("./sant-train.csv")

infile.columns.values
infile.TARGET.value_counts()

colcard = list()

for i in xrange(len(infile.columns)):
    colcard.append(len(infile.iloc[:,i].value_counts()))


In [3]:
colnz = infile.astype(bool).sum(axis=0)

len(colnz[colnz>50])

infile['saldo_medio_var13_medio_hace3'].describe()

infile.iloc[:,1:10].describe()

infile1 = infile.where(infile>0,0)
colnz = infile1.astype(bool).sum(axis=0)
collist = list(colnz[colnz>50].index)
infile2 = infile1.loc[:,collist]

In [4]:
shape(infile2)


(76020, 227)

In [5]:
trainfeat = infile2.drop(['TARGET'], axis=1)
trainlab = infile['TARGET']

X_train, X_test, y_train, y_test = cross_validation.train_test_split(trainfeat, trainlab, test_size=0.20, random_state=777)

X_train.shape, y_train.shape

#X_test.shape, y_test.shape

((60816, 226), (60816,))

In [7]:
clf = RandomForestClassifier(n_estimators=500,n_jobs=4,)
clf.fit(X_train,y_train)
#clf.score(X_test, y_test, scoring="roc_auc")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
scores = cross_validation.cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=5)



In [ ]:
#let's try normalization of parameters

#remove cols with <50 values


# #psuedo code:
# for i in n_cols
#     1. swap out -99999999 and other odd vals
#     2. scale vars

#try booleans
# for i in n_cols
#     1. swap out -99999999 and other odd vals
#     2. if val > 0, mark as 1

#look up text classif algos/sparse
infile.columns

In [23]:
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20

CV_rfc = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, scoring="roc_auc")
CV_rfc.fit(X_train, y_train)
print CV_rfc.best_params_

#{'bootstrap': True, 'min_samples_leaf': 4, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': 8, 'max_depth': None}


{'bootstrap': True, 'min_samples_leaf': 4, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': 8, 'max_depth': None}


In [24]:
CV_rfc.grid_scores_

[mean: 0.78210, std: 0.01099, params: {'bootstrap': False, 'min_samples_leaf': 9, 'min_samples_split': 1, 'criterion': 'entropy', 'max_features': 5, 'max_depth': 3},
 mean: 0.78058, std: 0.01411, params: {'bootstrap': True, 'min_samples_leaf': 6, 'min_samples_split': 8, 'criterion': 'gini', 'max_features': 8, 'max_depth': 3},
 mean: 0.76025, std: 0.01719, params: {'bootstrap': False, 'min_samples_leaf': 3, 'min_samples_split': 8, 'criterion': 'entropy', 'max_features': 1, 'max_depth': 3},
 mean: 0.81919, std: 0.00768, params: {'bootstrap': True, 'min_samples_leaf': 5, 'min_samples_split': 5, 'criterion': 'gini', 'max_features': 9, 'max_depth': None},
 mean: 0.78044, std: 0.01088, params: {'bootstrap': False, 'min_samples_leaf': 8, 'min_samples_split': 9, 'criterion': 'gini', 'max_features': 6, 'max_depth': 3},
 mean: 0.78055, std: 0.01142, params: {'bootstrap': False, 'min_samples_leaf': 10, 'min_samples_split': 10, 'criterion': 'gini', 'max_features': 8, 'max_depth': 3},
 mean: 0.7707

In [26]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=777)

gbfit = gbc.fit(X_train, y_train)
gbfit.score(X_test, y_test)



0.9607997895290713

In [34]:
param_dist_gbc = {"learning_rate": np.random.random_sample(1),
              "max_features": sp_randint(1, 11),
              "n_estimators": [20,50,100,200],
              "max_depth": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11)}

# run randomized search
n_iter_search = 20

CV_gbc = RandomizedSearchCV(gbc, param_distributions=param_dist_gbc,
                                   n_iter=n_iter_search, scoring="roc_auc")
CV_gbc.fit(X_train, y_train)
print CV_gbc.best_params_
#{'max_features': 10, 'n_estimators': 200, 'learning_rate': 0.013609745065371048, 'max_depth': 6, 'min_samples_leaf': 6}

{'max_features': 10, 'n_estimators': 200, 'learning_rate': 0.013609745065371048, 'max_depth': 6, 'min_samples_leaf': 6}


In [ ]:
np.random.random_sample(1)